In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from random import uniform, randint

In [ ]:
def mmc(potential, size, x_init, beta=1.0, step=0.5):
    """A simple Metropolis Monte Carlo implementation
    
    This function updates one particle at a time with a
    uniformly distributed step in each dimension.

    Arguments:
        potential (reference): potential function
        size (int): length of the sampled Markov chain
        x_init (numpy.ndarray): initial configuration
        beta (float): inverse temperature factor
        step (float): maximal step size
    """
    x = np.zeros((size, x_init.shape[0], x_init.shape[1]))
    u = np.zeros(size)
    x[0, :, :] = x_init
    u[0] = potential(x_init)
    x_ = np.zeros_like(x_init)
    for i in range(1, size):
        x_[:, :] = x[i - 1]
        j = randint(1, len(x_) - 2)
        x_[j] += np.random.uniform(-step, step, x_[j].shape)
        u_ = potential(x_)
        if u_ <= u[i - 1] or uniform(0, 1) < np.exp(beta * (u[i - 1] - u_)):
            x[i, :, :], u[i] = x_, u_
        else:
            x[i, :, :], u[i] = x[i - 1], u[i - 1]
    return x, u

In [ ]:
def gravity(x, g=9.81):
    """A model for gravity which penalizes x[:, 1]"""
    if np.any(x[:, 1] < 0.0):
        return np.inf
    return g * x[:, 1].sum()


def springs(x, k=50.0):
    """A pairwise spring potential"""
    return np.power(x[1:] - x[:-1], 2).sum() * k


def both(x, g=9.81, k=50.0):
    """This encapsulates gravity and spring"""
    return gravity(x, g=g) + springs(x, k=k)


x_init = 20 * np.ones((30, 2))
x_init[:, 0] = np.linspace(0, len(x_init) - 1, len(x_init))

x, u = mmc(both, 100000, x_init, beta=5, step=0.1)

fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(u)
ax.set_xlabel('time / steps')
ax.set_ylabel('energy / a.u.')
fig.tight_layout()

In [ ]:
fig, axes = plt.subplots(
    1, 2, figsize=(12, 5), sharex=True, sharey=True)
axes[0].plot(*x[0].T, '-s', label='initial')
axes[0].plot(*x[-1].T, '-o', label='final')
axes[0].legend()
for i in range(1, len(x_init) - 1):
    axes[1].plot(*x[::10, i, :].T)
axes[1].plot(*x[-1].T, '--', color='grey')
for ax in axes.flat:
    ax.set_xlabel(r'$x$')
axes[0].set_ylabel(r'$y$')
fig.tight_layout()